In [1]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random

In [2]:
#HTML('<script src="https://d3js.org/d3.v4.min.js"></script>')
filename_to_render = "SACPP_2021_10_20_17_44"

In [3]:
# JSON file
f = open (filename_to_render+'.json', "r")
  
# Reading from file
data = json.loads(f.read())

In [4]:
css_text = '''
.node circle {
  fill: #999;
}

.node text {
  font: 10px sans-serif;
}

.node--internal circle {
  fill: #555;
}

.node--internal text {
  text-shadow: 0 1px 0 #fff, 0 -1px 0 #fff, 1px 0 0 #fff, -1px 0 0 #fff;
}

.link {
  fill: none;
  stroke: #555;
  stroke-opacity: 0.4;
  stroke-width: 1.5px;
}

form {
  font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
  position: absolute;
  left: 10px;
  top: 10px;
}

label {
  display: block;
}

'''

In [5]:
js_text_template = Template('''

    var svg = d3.select("svg"),
        width = +svg.attr("width"),
        height = +svg.attr("height"),
        g = svg.append("g").attr("transform", "translate(40,0)");
    
    var tree = d3.tree()
        .size([height - 400, width - 160]);
    
    var cluster = d3.cluster()
        .size([height, width - 160]);
    
    var stratify = d3.stratify()
        .parentId(function(d) { return d.data.name });
        
// d3.csv("flare.csv", function(error, data) {
//   if (error) throw error;
  var data = $python_data

  var root = d3.hierarchy(data)
        .sort((a, b) => d3.descending(a.height, b.height) || d3.ascending(a.data.name, b.data.name));
    
        cluster(root);

        var link = g.selectAll(".link")
            .data(root.descendants().slice(1))
            .enter().append("path")
            .attr("class", "link")
            .attr("d", diagonal);

        var node = g.selectAll(".node")
            .data(root.descendants())
            .enter().append("g")
            .attr("class", function(d) { return "node" + (d.children ? " node--internal" : " node--leaf"); })
            .attr("transform", function(d) { return "translate(" + d.y + "," + d.x + ")"; });
    
        node.append("circle")
            .attr("r", 2.5);
    
        node.append("text")
            .attr("dy", 3)
            .attr("x", function(d) { return d.children ? -8 : 8; })
            .style("text-anchor", function(d) { return d.children ? "end" : "start"; })
            .text(function(d) { return d.data.name; });
    
        d3.selectAll("input")
      .on("change", changed);

      var timeout = setTimeout(function() {
         d3.select("input[value=\\"tree\\"]")
            .property("checked", true)
            .dispatch("change");
      }, 1000);
    
      function changed() {
        timeout = clearTimeout(timeout);
        (this.value === "tree" ? tree : cluster)(root);
        var t = d3.transition().duration(750);
        node.transition(t).attr("transform", function(d) { return "translate(" + d.y + "," + d.x + ")"; });
        link.transition(t).attr("d", diagonal);
        }

      function diagonal(d) {
        return "M" + d.y + "," + d.x
            + "C" + (d.parent.y + 100) + "," + d.x
            + " " + (d.parent.y + 100) + "," + d.parent.x
            + " " + d.parent.y + "," + d.parent.x;
        }
//});

''')

In [6]:
html_template = Template('''
<style> $css_text </style>
<form>
  <label><input type="radio" name="mode" value="cluster" checked> Dendrogram</label>
  <label><input type="radio" name="mode" value="tree"> Tree</label>
</form>
<svg width="960" height="2400"></svg>
<script src="https://d3js.org/d3.v4.min.js"></script>
<script> $js_text </script>
''')

In [7]:
js_text = js_text_template.substitute({'python_data': data})
#HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))

In [8]:
#data = HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text})).data

In [9]:
with open(filename_to_render+'.html', 'w') as outf:
    outf.write(html_template.substitute({'css_text': css_text, 'js_text': js_text}))
outf.close()

In [10]:
import webbrowser,os
webbrowser.open('file://' + os.path.realpath(filename_to_render+'.html'))

False